## Install the following libraries, if not done already, before running the below script

#### !pip install tensorflow
#### !pip install scikit-learn
#### !pip install numpy 
#### !pip install pandas 
#### !pip install pyDOE 
#### !pip install scipy
#### !pip install matplotlib
#### !pip install pypickle

In [ ]:
import numpy as np
import pandas as pd
from pyDOE import lhs  # Latin Hypercube Sampling
from scipy.optimize import newton
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from matplotlib import pyplot as plt
import os
from sklearn.model_selection import GroupShuffleSplit
import pickle

# Ensure reproducibility
np.random.seed(42)
tf.random.set_random_seed(42)


# Material constant ranges from Table 1
material_constant_ranges = {
    "E": [191000e6, 195000e6],  # Young's modulus in MPa
    "sigma_y": [90e6, 120e6],  # Yield stress in MPa
    "h": [5,10],  # Isotropic hardening rate
}
# Generate material constants using Latin Hypercube Sampling
def generate_material_constants_lhs(num_samples, ranges):
    num_variables = len(ranges)
    lhs_samples = lhs(num_variables, samples=num_samples)
    material_constants = []

    for i, (key, (low, high)) in enumerate(ranges.items()):
        samples = lhs_samples[:, i] * (high - low) + low
        material_constants.append(samples)

    return np.column_stack(material_constants)



def generate_strain_history(lb, ub):

    strain_history = np.zeros(3900)
    cumulative_strain = 0  # Start with zero cumulative strain

    # First range: Accumulate small positive strain increments
    for i in range(300):
        increment = 0.0001
        cumulative_strain += increment
        strain_history[i] = cumulative_strain

    # Second range: Accumulate negative strain increments
    for i in range(300, 900):
        increment = 0.0001
        cumulative_strain -= increment
        strain_history[i] = cumulative_strain

    # Third range: Return to positive strain increments
    for i in range(900, 1600):
        increment = 0.0001
        cumulative_strain += increment
        strain_history[i] = cumulative_strain
    
    for i in range(1600, 2300):
        increment = 0.0001
        cumulative_strain -= increment
        strain_history[i] = cumulative_strain
    
    for i in range(2300, 3100):
        increment = 0.0001
        cumulative_strain += increment
        strain_history[i] = cumulative_strain
        
    for i in range(3100, 3900):
        increment = 0.0001
        cumulative_strain -= increment
        strain_history[i] = cumulative_strain

    return strain_history
    


# Hooke's law (elastic predictor)
def elastic_predictor(eps, eps_p, E, h, alpha, sigma_y):
    sigma_trial = E * (eps - eps_p)  # Trial stress
    yield_function = np.abs(sigma_trial) - (sigma_y + h * alpha)
    return sigma_trial, yield_function


# Von Mises return mapping algorithm
def von_mises(eps, eps_p, sigma_trial, yield_function, alpha, h, E, sigma_y):
    if yield_function <= 0:
        # Elastic step
        sigma_updated = sigma_trial
        eps_p = eps_p
        alpha = alpha
        delta_gamma=0
        sign = sigma_trial/np.abs(sigma_trial)
    else:
        # Plastic step
        delta_gamma = yield_function / (E + h)
        eps_p = eps_p + delta_gamma* (sigma_trial/np.abs(sigma_trial))
        alpha = alpha + delta_gamma
        #sigma_updated = sigma_trial - E * delta_gamma
        sigma_updated = E*(eps-eps_p)
        sign = sigma_trial/np.abs(sigma_trial)

    return sigma_updated, eps_p, alpha, delta_gamma, sign



# Generate dataset
def generate_dataset(num_samples,num_strain_samples,lb, ub,ranges):
    
    dataset = []
    material_constants = generate_material_constants_lhs(num_samples, ranges)
    #print(material_constants)
    
    for constants in material_constants:
        
        E, sigma_y, h = constants
        strain_history = generate_strain_history(lb, ub)

        # Initialize state variables
        alpha, eps_p = 0, 0
    
        for eps in strain_history:
            # Elastic predictor
            sigma_trial, yield_function = elastic_predictor(eps, eps_p, E, h, alpha, sigma_y)

            # Return mapping algorithm
            sigma_updated, eps_p, alpha, delta_gamma, sign  = von_mises(eps, eps_p, sigma_trial, yield_function, alpha, h, E, sigma_y)

            # Store the data
            dataset.append([
                E,sigma_y,h, yield_function, eps, sigma_updated, eps_p, alpha, sign, np.abs(delta_gamma)
            ])

    return np.array(dataset)


# Save the dataset to a CSV file
def save_dataset_to_csv(dataset, filename="generated_dataset_von_mises_training_07_01_2025.csv"):
    columns = ["E","sigma_y","h", "yield_function", "strain", "sigma_updated", "cummulative eps_p", "alpha",'sign', 'plastic_strain']
    df = pd.DataFrame(dataset, columns=columns)
    df.to_csv(filename, index=False)
    print(f"Dataset saved to {filename}")


In [2]:
def normalize_dataset(data, scaler=None):
    if scaler is None:
        scaler = MinMaxScaler()
        normalized_data = scaler.fit_transform(data)
    else:
        normalized_data = scaler.transform(data)
    return normalized_data, scaler

In [3]:
# Build the neural network
def build_model(input_dim):
    model = Sequential([
        Dense(10, activation='sigmoid', input_shape=(input_dim,)),  # Use 10 neurons, sigmoid activation
        Dense(1, activation='linear')     # Linear activation for the output
    ])
    custom_adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(optimizer=custom_adam, loss='mean_squared_error', metrics=['mae'])
    return model

In [4]:
# Plot training and validation loss
def plot_loss(history, filename='loss_curve.png'):
    plt.figure(figsize=(8, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss (MSE)')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    #plt.savefig(filename)  # Save the plot as an image file
    #plt.close()  # Close the plot to avoid displaying it in the notebook
    #print(f"Loss curve saved as {filename}")

# Plot predicted vs original values
def plot_predicted_vs_actual(y_test, y_pred, filename='predicted_vs_actual.png'):
    plt.figure(figsize=(8, 6))
    plt.plot(y_test, label='Original Values', color='blue')
    plt.plot(y_pred, label='Predicted Values', color='red', linestyle='--')
    plt.xlabel('Sample index')
    plt.ylabel('Plastic Strain')
    plt.title('Predicted vs. Original Plastic Strain')
    plt.legend()
    plt.grid(True)
    plt.show()
    plt.savefig('predicted_vs_actual')  # Save the plot as an image file
    plt.close()  # Close the plot to avoid displaying it in the notebookLHSLHS
    print(f"Plot saved as {'predicted_vs_actual'}")

In [5]:
training = generate_dataset(num_samples=500, num_strain_samples=1500, lb=0.0001,ub=0.0002, ranges=material_constant_ranges)

In [6]:
columns = ["E","sigma_y","h", "yield_function", "strain", "sigma_updated", "cumulative_plastic_strain", "alpha", 'sign','plastic_strain']
df = pd.DataFrame(training, columns=columns)

In [7]:
df.head(50)

,E,sigma_y,h,yield_function,strain,sigma_updated,cumulative_plastic_strain,alpha,sign,plastic_strain
0,1.933522e+11,9.544861e+07,7.982931,-7.611339e+07,0.0001,1.933522e+07,0.000000,0.000000,1.0,0.000000
1,1.933522e+11,9.544861e+07,7.982931,-5.677818e+07,0.0002,3.867043e+07,0.000000,0.000000,1.0,0.000000
2,1.933522e+11,9.544861e+07,7.982931,-3.744296e+07,0.0003,5.800565e+07,0.000000,0.000000,1.0,0.000000
3,1.933522e+11,9.544861e+07,7.982931,-1.810774e+07,0.0004,7.734086e+07,0.000000,0.000000,1.0,0.000000
4,1.933522e+11,9.544861e+07,7.982931,1.227472e+06,0.0005,9.544861e+07,0.000006,0.000006,1.0,0.000006
5,1.933522e+11,9.544861e+07,7.982931,1.933522e+07,0.0006,9.544861e+07,0.000106,0.000106,1.0,0.000100
6,1.933522e+11,9.544861e+07,7.982931,1.933522e+07,0.0007,9.544861e+07,0.000206,0.000206,1.0,0.000100
7,1.933522e+11,9.544861e+07,7.982931,1.933522e+07,0.0008,9.544861e+07,0.000306,0.000306,1.0,0.000100
8,1.933522e+11,9.544861e+07,7.982931,1.933522e+07,0.0009,9.544861e+07,0.000406,0.000406,1.0,0.000100
9,1.933522e+11,9.544861e+07,7.982931,1.933522e+07,0.0010,9.544861e+07,0.000506,0.000506,1.0,0.000100


In [8]:
df=df.fillna(1)

In [9]:
df.isna().sum()

E                            0
sigma_y                      0
h                            0
yield_function               0
strain                       0
sigma_updated                0
cumulative_plastic_strain    0
alpha                        0
sign                         0
plastic_strain               0
dtype: int64

In [10]:
df.shape

(1950000, 10)

In [11]:
# Separate features and target
X = df.iloc[:, :4]  # Features (E, sigma_y, c1, c2, c3, gamma1, gamma2, gamma3, b, Q, strain)
y = df.iloc[:, -1]   # Targets (plastic_strain)

In [12]:
training[:, -1] 

array([0.e+00, 0.e+00, 0.e+00, ..., 1.e-04, 1.e-04, 1.e-04])

In [13]:
X_normalized, scaler_X= normalize_dataset(X)
y = y.values.reshape(-1, 1)
y_normalized, scaler_y= normalize_dataset(y)

In [14]:
# Build the neural network model
print("Building the neural network model...")
model = build_model(X.shape[1])

# Define the early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',         # Monitor the validation loss
    patience=20,                # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True   # Restore model weights from the epoch with the best value of the monitored quantity
)

# Train the model with early stopping
print("Training the model...")
history = model.fit(
    X_normalized, 
    y_normalized, 
    epochs=3000,               # Set a large number of epochs
    batch_size=1000,
    validation_split=0.2, 
    callbacks=[early_stopping],  # Include the early stopping callback
    verbose=1
)

Building the neural network model...
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Training the model...
Train on 1560000 samples, validate on 390000 samples
Epoch 1/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 0.0222 - mean_absolute_error: 0.0567 - val_loss: 0.0115 - val_mean_absolute_error: 0.0256
Epoch 2/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 0.0103 - mean_absolute_error: 0.0242 - val_loss: 0.0087 - val_mean_absolute_error: 0.0232
Epoch 3/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 0.0066 - mean_absolute_error: 0.0189 - val_loss: 0.0043 - val_mean_absolute_error: 0.0144
Epoch 4/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 0.0030 - mean_absolute_error: 0.0110 - val_loss: 0.0024 - val_mean_absolute_error: 0.0112
Epoch 5/3000
1560000/1560000 [==============================] - 1s

1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.3283e-04 - mean_absolute_error: 0.0018 - val_loss: 1.3177e-04 - val_mean_absolute_error: 0.0023
Epoch 88/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.3138e-04 - mean_absolute_error: 0.0018 - val_loss: 1.2999e-04 - val_mean_absolute_error: 0.0016
Epoch 89/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.2973e-04 - mean_absolute_error: 0.0018 - val_loss: 1.2781e-04 - val_mean_absolute_error: 0.0016
Epoch 90/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.2837e-04 - mean_absolute_error: 0.0018 - val_loss: 1.2736e-04 - val_mean_absolute_error: 0.0018
Epoch 91/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.2676e-04 - mean_absolute_error: 0.0018 - val_loss: 1.2575e-04 - val_mean_absolute_error: 0.0021
Epoch 92/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.

Epoch 175/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 4.0462e-05 - mean_absolute_error: 0.0011 - val_loss: 4.0455e-05 - val_mean_absolute_error: 8.0931e-04
Epoch 176/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 3.9765e-05 - mean_absolute_error: 0.0011 - val_loss: 4.1214e-05 - val_mean_absolute_error: 0.0018
Epoch 177/3000
1560000/1560000 [==============================] - 2s 1us/sample - loss: 3.9045e-05 - mean_absolute_error: 0.0010 - val_loss: 3.9843e-05 - val_mean_absolute_error: 0.0014
Epoch 178/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 3.8358e-05 - mean_absolute_error: 0.0010 - val_loss: 3.8649e-05 - val_mean_absolute_error: 0.0012
Epoch 179/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 3.7802e-05 - mean_absolute_error: 0.0011 - val_loss: 3.7711e-05 - val_mean_absolute_error: 7.6599e-04
Epoch 180/3000
1560000/1560000 [==============================

Epoch 218/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 2.4572e-05 - mean_absolute_error: 9.5049e-04 - val_loss: 2.4350e-05 - val_mean_absolute_error: 6.5331e-04
Epoch 219/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 2.4375e-05 - mean_absolute_error: 9.1800e-04 - val_loss: 2.4734e-05 - val_mean_absolute_error: 0.0012
Epoch 220/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 2.4256e-05 - mean_absolute_error: 9.4752e-04 - val_loss: 2.3915e-05 - val_mean_absolute_error: 5.8600e-04
Epoch 221/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 2.4085e-05 - mean_absolute_error: 9.3168e-04 - val_loss: 2.3764e-05 - val_mean_absolute_error: 5.7822e-04
Epoch 222/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 2.3978e-05 - mean_absolute_error: 9.5432e-04 - val_loss: 2.3805e-05 - val_mean_absolute_error: 9.0351e-04
Epoch 223/3000
1560000/1560000 [==

1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.7601e-05 - mean_absolute_error: 8.3305e-04 - val_loss: 1.7383e-05 - val_mean_absolute_error: 7.8296e-04
Epoch 303/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.7518e-05 - mean_absolute_error: 8.0840e-04 - val_loss: 1.7153e-05 - val_mean_absolute_error: 5.3345e-04
Epoch 304/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.7519e-05 - mean_absolute_error: 8.5219e-04 - val_loss: 1.7704e-05 - val_mean_absolute_error: 0.0011
Epoch 305/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.7437e-05 - mean_absolute_error: 8.1904e-04 - val_loss: 1.7042e-05 - val_mean_absolute_error: 5.0531e-04
Epoch 306/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.7411e-05 - mean_absolute_error: 8.2647e-04 - val_loss: 1.7058e-05 - val_mean_absolute_error: 6.3535e-04
Epoch 307/3000
1560000/1560000 [=================

1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.4824e-05 - mean_absolute_error: 7.9676e-04 - val_loss: 1.4344e-05 - val_mean_absolute_error: 4.5551e-04
Epoch 387/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.4748e-05 - mean_absolute_error: 7.6057e-04 - val_loss: 1.4913e-05 - val_mean_absolute_error: 0.0011
Epoch 388/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.4808e-05 - mean_absolute_error: 8.1612e-04 - val_loss: 1.4864e-05 - val_mean_absolute_error: 0.0011
Epoch 389/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.4768e-05 - mean_absolute_error: 8.1562e-04 - val_loss: 1.4368e-05 - val_mean_absolute_error: 6.6099e-04
Epoch 390/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.4726e-05 - mean_absolute_error: 7.9831e-04 - val_loss: 1.4560e-05 - val_mean_absolute_error: 9.0811e-04
Epoch 391/3000
1560000/1560000 [=====================

1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.3292e-05 - mean_absolute_error: 7.6188e-04 - val_loss: 1.2973e-05 - val_mean_absolute_error: 7.9395e-04
Epoch 471/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.3314e-05 - mean_absolute_error: 7.8625e-04 - val_loss: 1.2712e-05 - val_mean_absolute_error: 3.5793e-04
Epoch 472/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.3253e-05 - mean_absolute_error: 7.5130e-04 - val_loss: 1.3246e-05 - val_mean_absolute_error: 0.0010
Epoch 473/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.3280e-05 - mean_absolute_error: 7.8635e-04 - val_loss: 1.3901e-05 - val_mean_absolute_error: 0.0014
Epoch 474/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.3255e-05 - mean_absolute_error: 7.7280e-04 - val_loss: 1.2861e-05 - val_mean_absolute_error: 6.9998e-04
Epoch 475/3000
1560000/1560000 [=====================

1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.2442e-05 - mean_absolute_error: 7.4055e-04 - val_loss: 1.2245e-05 - val_mean_absolute_error: 9.2357e-04
Epoch 555/3000
1560000/1560000 [==============================] - 2s 1us/sample - loss: 1.2479e-05 - mean_absolute_error: 7.7768e-04 - val_loss: 1.1924e-05 - val_mean_absolute_error: 5.9040e-04
Epoch 556/3000
1560000/1560000 [==============================] - 2s 1us/sample - loss: 1.2436e-05 - mean_absolute_error: 7.5044e-04 - val_loss: 1.3288e-05 - val_mean_absolute_error: 0.0015
Epoch 557/3000
1560000/1560000 [==============================] - 2s 1us/sample - loss: 1.2439e-05 - mean_absolute_error: 7.4870e-04 - val_loss: 1.2037e-05 - val_mean_absolute_error: 7.3195e-04
Epoch 558/3000
1560000/1560000 [==============================] - 2s 1us/sample - loss: 1.2424e-05 - mean_absolute_error: 7.5011e-04 - val_loss: 1.1845e-05 - val_mean_absolute_error: 4.8407e-04
Epoch 559/3000
1560000/1560000 [=================

1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.1949e-05 - mean_absolute_error: 7.4137e-04 - val_loss: 1.1333e-05 - val_mean_absolute_error: 4.4697e-04
Epoch 639/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.1954e-05 - mean_absolute_error: 7.5727e-04 - val_loss: 1.1323e-05 - val_mean_absolute_error: 4.6050e-04
Epoch 640/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.1880e-05 - mean_absolute_error: 7.0963e-04 - val_loss: 1.1324e-05 - val_mean_absolute_error: 4.2665e-04
Epoch 641/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.1906e-05 - mean_absolute_error: 7.3270e-04 - val_loss: 1.1861e-05 - val_mean_absolute_error: 0.0010
Epoch 642/3000
1560000/1560000 [==============================] - 1s 1us/sample - loss: 1.1947e-05 - mean_absolute_error: 7.5244e-04 - val_loss: 1.3533e-05 - val_mean_absolute_error: 0.0018
Epoch 643/3000
1560000/1560000 [=====================

In [15]:
# Save the model
model.save('plastic_strain_predictor_model_von_mises_20_01_2025.h5')
#print("Model saved as 'plastic_strain_predictor_model.h5'")

with open('scaler_X_von_mises_20_01_2025.pkl', 'wb') as file:
    pickle.dump(scaler_X, file)
with open('scaler_y_von_mises_20_01_2025.pkl', 'wb') as file:
    pickle.dump(scaler_y, file)